In [13]:
library(limma)
library(edgeR)

filename_in <- '../../data/Flounder/DANRE-PAROL_liver.count.txt'
filename_base <- filename_in
sample1 <- 'DR'
sample2 <- 'PO'
min_cpm1 <- 2

x <- read.delim(filename_in,row.names='GeneName',header=T)
groups <- gsub('_[123]$','',colnames(x))
keep <- rowSums(cpm(x)>1) >= min_cpm1
x <- x[keep,]

y <- DGEList(counts=x,group=groups)
design <- model.matrix( ~ as.factor(groups) )

v <- voom(y, design)

fit <- lmFit(v, design)
fit <- eBayes(fit, trend=TRUE)

write.table( topTable(fit, coef=ncol(design), confint=TRUE, n=Inf), file='../../data/Flounder/DANRE-PAROL_liver.limma_out.txt' )



In [15]:
filename_in <- '../../data/Flounder/DANRE-PAROL_liver.count.txt'
filename_base <- filename_in
sample1 <- 'DR'
sample2 <- 'PO'
min_cpm1 <- 2

filename_base <- sub('.txt','',filename_in)
filename_cpm <- sub('.txt','.edgeR_cpm.txt',filename_in)
filename_out <- sub('.txt','.edgeR_out.txt',filename_in)


library(edgeR)

x <- read.delim(filename_in,row.names='GeneName',header=T)
groups <- gsub('_[123]$','',colnames(x))
keep <- rowSums(cpm(x)>1) >= min_cpm1
x <- x[keep,]

y <- DGEList(counts=x,group=groups)
y <- calcNormFactors(y)
y <- estimateCommonDisp(y)
y <- estimateTagwiseDisp(y)

write.table(cpm(x), file=filename_cpm, row.names=TRUE, col.names=TRUE, sep='\t')
write.table(topTags(exactTest(y,pair=c(sample2,sample1)),n=Inf), paste(filename_base,paste(sample1,sample2,sep='-'),'edgeR_out',sep='.'),sep='\t')
